# Testing

## Imports

In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

## Functions

In [2]:
def generate_random_rank_r_square_matrix_vector_b(m):
    r = int(np.floor(0.5 * m))
    while True:
        A = np.random.rand(m, m)
        U, S, VT = np.linalg.svd(A)
        S_bar = np.zeros((m, m))
        if S.shape[0] < r:
            continue
        for i in range(r):
            S_bar[i, i] = S[i]
        A = np.dot(U, np.dot(S_bar, VT))
        coefficients = np.random.rand(A.shape[1])
        b = np.dot(A, coefficients)
        return A, b

In [3]:
def problem_1_norm_PN_viable_solution(A, H, b, lambda_values, m):
    epsilon = 10 ** -5
    AHb = np.dot(A, np.dot(H, b))
    for i in range(m):
        if np.abs(AHb[i] - b[i]) > epsilon:
            print("AHb - b == 0 violation")
            return False
        if lambda_values[i] < -epsilon:
            print("lambda >= 0 violation")
            return False
        if np.abs(lambda_values[i] * (AHb[i] - b[i])) > epsilon:
            print("lambda^T * (AHb - b) == 0 violation")
            return False
    return True

In [4]:
def problem_1_norm_P1_P4_viable_solution(A, H, m):
    epsilon = 10 ** -5
    AHA = np.dot(A, np.dot(H, A))
    for i in range(m):
        for j in range(m):
            if np.abs(AHA[i, j] - A[i, j]) > epsilon:
                return False
    HA_T = np.dot(H, A).T
    HA = np.dot(H, A)
    for i in range(m):
        for j in range(m):
            if np.abs(HA_T[i, j] - HA[i, j]) > epsilon:
                return False
    return True

In [5]:
def problem_1_norm_MSN_viable_solution(A, H, m):
    epsilon = 10 ** -5
    HAA_T = np.dot(H, np.dot(A, A.T))
    A_T = A.T
    for i in range(m):
        for j in range(m):
            if np.abs(HAA_T[i, j] - A_T[i, j]) > epsilon:
                return False
    return True

In [31]:
def problem_2_norm_KKT_solver(A, b):
    # Extracts matrix dimensions
    m = A.shape[0]

    # Creates a new model
    model = gp.Model("nlp")

    # Creates variables
    H_var = model.addVars(m, m, lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="H")
    H = []
    for i in range(m):
        H_row = []
        for j in range(m):
            H_row.append(H_var[i, j])
        H.append(H_row)
    H = np.array(H)

    # Defines objective function
    objective = np.dot(np.dot(H, b).T, np.dot(H, b))

    # Sets objective function
    model.setObjective(objective, GRB.MINIMIZE)

    # Adds constraint: AHb = b
    AHb = np.dot(A, np.dot(H, b))
    for i in range(m):
        model.addConstr(AHb[i] == b[i], name=f"constraint_eq_i={i}")

    # Configures log to not show messages on terminal
    model.setParam("OutputFlag", 0)

    # Defines SoftMemLimit (in GigaBytes)
    # model.setParam('SoftMemLimit', 8)

    # Makes Gurobi compute the dual multipliers for continuous convex QCPs by solving the corresponding KKT problem
    # model.setParam("QCPDual", 1)
    model.setParam("ConcurrentMethod", 1)

    # Optimizes model
    model.optimize()

    # Checks if model was optimized successfully
    if model.status == GRB.OPTIMAL:
        # Extracts model variables
        lambda_values = []
        for i in range(m):
            lambda_value = model.getConstrByName(f"constraint_eq_i={i}").Pi
            lambda_values.append(lambda_value)
        lambda_values = np.array(lambda_values)
        return lambda_values
    else:
        raise RuntimeError(f"Model was not optimized successfully. Status: {model.status}")

In [7]:
def problem_1_norm_PN_solver(A, b):
    # Solves the following problem to get the lagrange multipliers for the PN equations
    lambda_values = problem_2_norm_KKT_solver(A=A, b=b)

    # Extracts matrix dimensions
    m = A.shape[0]

    # Defines auxiliary variables
    J = np.ones((m, m))

    # Creates a new model
    model = gp.Model("lp")

    # Creates variables
    H_var = model.addVars(m, m, lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="H")
    Z_var = model.addVars(m, m, lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="Z")
    H = []
    Z = []
    for i in range(m):
        H_row = []
        Z_row = []
        for j in range(m):
            H_row.append(H_var[i, j])
            Z_row.append(Z_var[i, j])
        H.append(H_row)
        Z.append(Z_row)
    H = np.array(H)
    Z = np.array(Z)

    # Defines objective function
    objective = np.trace(np.dot(J.T, Z))

    # Sets objective function
    model.setObjective(objective, GRB.MINIMIZE)

    # Adds constraint: Primal feasibility AHb = b
    AHb = np.dot(A, np.dot(H, b))
    for i in range(m):
        model.addConstr(AHb[i] == b[i], name=f"constraint_eq_1_i={i}")

    # Adds constraint: Complementary slackness lambda^T * (AHb - b) = 0
    for i in range(m):
        model.addConstr(lambda_values[i] * (AHb[i] - b[i]) == 0, name=f"constraint_eq_2_i={i}")

    # Adds constraint: Z - H >= 0
    Z_minus_H = Z - H
    for i in range(m):
        for j in range(m):
            model.addConstr(Z_minus_H[i, j] >= 0, name=f"constraint_ineq_1_i={i}_j={j}")

    # Adds constraint: Z + H >= 0
    Z_plus_H = Z + H
    for i in range(m):
        for j in range(m):
            model.addConstr(Z_plus_H[i, j] >= 0, name=f"constraint_ineq_2_i={i}_j={j}")

    # Configures log to not show messages on terminal
    model.setParam("OutputFlag", 0)

    # Defines SoftMemLimit (in GigaBytes)
    # model.setParam('SoftMemLimit', 8)

    # Optimizes model
    model.optimize()

    # Checks if model was optimized successfully
    if model.status == GRB.OPTIMAL:
        # Extracts model variables
        H_values = np.zeros((m, m))
        for i in range(m):
            for j in range(m):
                H_values[i, j] = H[i, j].X
        return H_values
    else:
        raise RuntimeError(f"Model was not optimized successfully. Status: {model.status}")

In [8]:
def problem_1_norm_P1_P4_solver(A, b):
    # Extracts matrix dimensions
    m = A.shape[0]

    # Defines auxiliary variables
    J = np.ones((m, m))

    # Creates a new model
    model = gp.Model("lp")

    # Creates variables
    H_var = model.addVars(m, m, lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="H")
    Z_var = model.addVars(m, m, lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="Z")
    H = []
    Z = []
    for i in range(m):
        H_row = []
        Z_row = []
        for j in range(m):
            H_row.append(H_var[i, j])
            Z_row.append(Z_var[i, j])
        H.append(H_row)
        Z.append(Z_row)
    H = np.array(H)
    Z = np.array(Z)

    # Defines objective function
    objective = np.trace(np.dot(J.T, Z))

    # Sets objective function
    model.setObjective(objective, GRB.MINIMIZE)

    # Adds constraint: P1 given by AHA = A
    AHA = np.dot(A, np.dot(H, A))
    for i in range(m):
        for j in range(m):
            model.addConstr(AHA[i, j] == A[i, j], name=f"constraint_P1_i={i}_j={j}")

    # Adds constraint: P4 given by (HA)^T = HA
    HA_T = np.dot(H, A).T
    HA = np.dot(H, A)
    for i in range(m):
        for j in range(m):
            model.addConstr(HA_T[i, j] == HA[i, j], name=f"constraint_P3_i={i}_j={j}")

    # Adds constraint: Z - H >= 0
    Z_minus_H = Z - H
    for i in range(m):
        for j in range(m):
            model.addConstr(Z_minus_H[i, j] >= 0, name=f"constraint_ineq_1_i={i}_j={j}")

    # Adds constraint: Z + H >= 0
    Z_plus_H = Z + H
    for i in range(m):
        for j in range(m):
            model.addConstr(Z_plus_H[i, j] >= 0, name=f"constraint_ineq_2_i={i}_j={j}")

    # Configures log to not show messages on terminal
    model.setParam("OutputFlag", 0)

    # Defines SoftMemLimit (in GigaBytes)
    # model.setParam('SoftMemLimit', 8)

    # Optimizes model
    model.optimize()

    # Checks if model was optimized successfully
    if model.status == GRB.OPTIMAL:
        # Extracts model variables
        H_values = np.zeros((m, m))
        for i in range(m):
            for j in range(m):
                H_values[i, j] = H[i, j].X
        return H_values
    else:
        raise RuntimeError(f"Model was not optimized successfully. Status: {model.status}")

In [9]:
def problem_1_norm_MSN_solver(A, b):
    # Extracts matrix dimensions
    m = A.shape[0]

    # Defines auxiliary variables
    J = np.ones((m, m))

    # Creates a new model
    model = gp.Model("lp")

    # Creates variables
    H_var = model.addVars(m, m, lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="H")
    Z_var = model.addVars(m, m, lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="Z")
    H = []
    Z = []
    for i in range(m):
        H_row = []
        Z_row = []
        for j in range(m):
            H_row.append(H_var[i, j])
            Z_row.append(Z_var[i, j])
        H.append(H_row)
        Z.append(Z_row)
    H = np.array(H)
    Z = np.array(Z)

    # Defines objective function
    objective = np.trace(np.dot(J.T, Z))

    # Sets objective function
    model.setObjective(objective, GRB.MINIMIZE)

    # Adds constraint: MSN given by HAA^T = A^T
    HAA_T = np.dot(H, np.dot(A, A.T))
    A_T = A.T
    for i in range(m):
        for j in range(m):
            model.addConstr(HAA_T[i, j] == A_T[i, j], name=f"constraint_MSN_i={i}_j={j}")

    # Adds constraint: Z - H >= 0
    Z_minus_H = Z - H
    for i in range(m):
        for j in range(m):
            model.addConstr(Z_minus_H[i, j] >= 0, name=f"constraint_ineq_1_i={i}_j={j}")

    # Adds constraint: Z + H >= 0
    Z_plus_H = Z + H
    for i in range(m):
        for j in range(m):
            model.addConstr(Z_plus_H[i, j] >= 0, name=f"constraint_ineq_2_i={i}_j={j}")

    # Configures log to not show messages on terminal
    model.setParam("OutputFlag", 0)

    # Defines SoftMemLimit (in GigaBytes)
    # model.setParam('SoftMemLimit', 8)

    # Optimizes model
    model.optimize()

    # Checks if model was optimized successfully
    if model.status == GRB.OPTIMAL:
        # Extracts model variables
        H_values = np.zeros((m, m))
        for i in range(m):
            for j in range(m):
                H_values[i, j] = H[i, j].X
        return H_values
    else:
        raise RuntimeError(f"Model was not optimized successfully. Status: {model.status}")

In [10]:
m = 80
A, b = generate_random_rank_r_square_matrix_vector_b(m=m)

In [32]:
lambda_values = problem_2_norm_KKT_solver(A=A, b=b)

In [ ]:
print(lambda_values)

In [ ]:
H_star = problem_1_norm_PN_solver(A=A, b=b)
print(problem_1_norm_PN_viable_solution(A=A, H=H_star, b=b, lambda_values=lambda_values, m=m))
print(np.linalg.norm(H_star.flatten(), ord=1))
print(H_star)

In [ ]:
# Usando numpy.nonzero() para encontrar os índices dos elementos não nulos
indices = np.nonzero(H_star)

# Imprimindo os elementos não nulos
elementos_nao_nulos = H_star[indices]
print(elementos_nao_nulos)
print(elementos_nao_nulos.shape)
print(indices[1])

In [ ]:
H_star = problem_1_norm_P1_P4_solver(A=A, b=b)
print(problem_1_norm_P1_P4_viable_solution(A=A, H=H_star, m=m))
print(np.linalg.norm(H_star.flatten(), ord=1))
print(H_star)

In [ ]:
H_star = problem_1_norm_MSN_solver(A=A, b=b)
print(problem_1_norm_MSN_viable_solution(A=A, H=H_star, m=m))
print(np.linalg.norm(H_star.flatten(), ord=1))
print(H_star)